#### In this notebook we build a Web Crawler to visitit and save the content of pages inside the uic domain:

In [1]:
from html.parser import HTMLParser  
from urllib.request import urlopen  
from urllib import parse
# import libraries
#import urllib2
from bs4 import BeautifulSoup
import PyPDF2
import requests
import io

# We are going to create a class called LinkParser that inherits some
# methods from HTMLParser which is why it is passed into the definition
class LinkParser(HTMLParser):
    already_visited = []
    
    #remove '/' at the end of the url
    def normalizeUrl(url):
        #print("ciao4")
        if url.endswith('/'):
            url = url[:-1]
            #print("trimmed")
        return url
    
    # This is a function that HTMLParser normally has
    # but we are adding some functionality to it
    def handle_starttag(self, tag, attrs):
        # We are looking for the begining of a link. Links normally look
        # like <a href="www.someurl.com"></a>
        if tag == 'a':
            for (key, value) in attrs:
                if key == 'href':
                    # We are grabbing the new URL. We are also adding the
                    # base URL to it. For example:
                    # www.netinstructions.com is the base and
                    # somepage.html is the new URL (a relative URL)
                    #
                    # We combine a relative URL with the base URL to create
                    # an absolute URL like:
                    # www.netinstructions.com/somepage.html
                    newUrl = parse.urljoin(self.baseUrl, value)
                    
                    #check if url contains an hashtag i.e .uic/#artificialinteligence
                    if ("#" in newUrl):
                        #we have to take off the hashtag to normalize the link
                        pos = newUrl.find("#")
                        newUrl=newUrl[:pos]
                        #print("hashtag!")
                        #print(newUrl)
                    # And add it to our colection of links:
                    
            
                    if str(newUrl).endswith('/'):
                        newUrl = str(newUrl)[:-1]
                        
                    #when url starts with https
                    #check if the same url but with http is not already present
                    normUrl = newUrl
                    flag = False
                    if ("https" in newUrl):
                        #find first occurence of s (in https:)
                        pos = newUrl.find("s")
                        normUrl = newUrl[:pos] + newUrl[pos+1:]
                        flag = True
                        #print("https!!")
                        #print(normUrl)
                        # print(newUrl)
                            
                    if not(newUrl in self.already_visited) and not(normUrl in self.already_visited)  and ('.uic.edu' in newUrl):
                        self.already_visited.append(newUrl)
                        if (flag):
                            self.already_visited.append(normUrl)
                
                        self.links = self.links + [newUrl]

    # This is a function to get links
    # that the spider() function will call
    def getLinks(self, url):
        self.links = []
        # Remember the base URL which will be important when creating
        # absolute URLs
        self.baseUrl = url
        # Use the urlopen function from the standard Python 3 library
        response = urlopen(url)
        # Make sure that we are looking at HTML and not other things that
        # are floating around on the internet (such as
        # JavaScript files, CSS, or .PDFs for example)
        print(response.getheader('Content-Type'))
        
        #check if it is a HTML page
        if 'text/html' in response.getheader('Content-Type'):
            htmlBytes = response.read()
            #html = urllib.urlopen(url).read()
            soup = BeautifulSoup(htmlBytes)
            #text=soup.body.replace('\n',' ')

            # kill all script and style elements
            for script in soup(["script", "style"]):
                script.extract()    # rip it out

            # get text
            text = soup.get_text()
            # break into lines and remove leading and trailing space on each
            lines = (line.strip() for line in text.splitlines())
            # break multi-headlines into a line each
            chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
            # drop blank lines
            text = '\n'.join(chunk for chunk in chunks if chunk)
            
            #check if the parsing success
            #print(text)
            
            # (A change from Python 2.x to Python 3.x)
            htmlString = htmlBytes.decode("utf-8")
            self.feed(htmlString)
            #print(self.links)
            return text, self.links
        else:
            if 'application/pdf' in response.getheader('Content-Type'):
                print("PDF FOUND!!!")
                string = ""
                r = requests.get(url)
                f = io.BytesIO(r.content)
                reader = PyPDF2.PdfFileReader(f)
                if reader.isEncrypted:
                    reader.decrypt('')
                contents =""
                for p in range(reader.getNumPages()):
                    contents += reader.getPage(p).extractText()
                    #print(contents)
                return contents,[]
                    
            return "-1",[]

   

In [2]:
 # And finally here is our spider. It takes in an URL, a word to find,
    # and the number of pages to search through before giving up
import codecs
import re
def spider(url,maxPages):  
    pagesToVisit = [url]
    numberVisited = 0
    parser = LinkParser()
    id = 0
    #dictionary id-> url of the web page
    urls_dictionary={}
    # The main loop. Create a LinkParser and get all the links on the page.
    # Also search the page for the word or string
    # In our getLinks function we return the web page
    # (this is useful for searching for the word)
    # and we return a set of links from that web page
    # (this is useful for where to go next)
    while numberVisited < maxPages and pagesToVisit != []:
        # Start from the beginning of our collection of pages to visit:
        url = pagesToVisit[0]
        parser.already_visited.append(url)
        #pop url
        pagesToVisit = pagesToVisit[1:]
        try:
            print(numberVisited, "Visiting:", url)
            #parser = LinkParser()
            data, links = parser.getLinks(url)
            #print(type(data))
            #print(data)
            
            #check if the url we just crawled is a url to crawl
            alpha=re.findall('[a-z]', data)
            if (data=="-1") or (len(alpha)<3):
                #print("URL NOT CRAWLED")
                continue
              
            ###write data into file
            with io.open("./documents2/" +str(id) +".txt", 'w', encoding="utf-8") as file:
                file.write(url)
                file.write("\n")
                file.write(data) 
                
            #update the list of pages to visit    
            pagesToVisit = pagesToVisit + links
            print("done " + str(numberVisited))
            #print(pagesToVisit)
            #add the entry id-> url in the dictionary
            urls_dictionary[id]=url
            id +=1
            numberVisited = numberVisited +1
        except Exception as e:
            print(e)
            print(" **Failed!**")

    return urls_dictionary

In [4]:
dictionary = spider("https://www.cs.uic.edu", 3200)

0 Visiting: https://www.cs.uic.edu
text/html; charset=utf-8
done 0
1 Visiting: http://engineering.uic.edu


C:\Users\pierv\AppData\Local\Continuum\anaconda3\lib\site-packages\bs4\__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file C:\Users\pierv\AppData\Local\Continuum\anaconda3\lib\runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))


text/html; charset=UTF-8
done 1
2 Visiting: http://www.engineering.uic.edu/donate
text/html; charset=UTF-8
done 2
3 Visiting: https://www.cs.uic.edu/undergraduate-admissions
text/html; charset=utf-8
done 3
4 Visiting: https://www.cs.uic.edu/visit-us-at-an-open-house
text/html; charset=utf-8
done 4
5 Visiting: https://www.cs.uic.edu/graduate-admissions
text/html; charset=utf-8
done 5
6 Visiting: https://www.cs.uic.edu/undergraduate-programs
text/html; charset=utf-8
done 6
7 Visiting: https://www.cs.uic.edu/graduate-programs
text/html; charset=utf-8
done 7
8 Visiting: https://catalog.uic.edu/ucat/colleges-depts/engineering/cs/joint-bs-ms
text/html; charset=UTF-8
done 8
9 Visiting: https://www.cs.uic.edu/cs-minor
text/html; charset=utf-8
done 9
10 Visiting: https://www.cs.uic.edu/courses
text/html; charset=utf-8
done 10
11 Visiting: https://www.cs.uic.edu/scholarships
text/html; charset=utf-8
done 11
12 Visiting: http://ecc.uic.edu/computer-science
text/html; charset=UTF-8
done 12
13 Visi

text/html; charset=utf-8
done 82
83 Visiting: http://engineering.uic.edu/2018/10/09/falling-out-of-love-with-science-aimbe-exec-presents-at-bioe-seminar-series
text/html; charset=UTF-8
done 83
84 Visiting: http://che.uic.edu
text/html; charset=UTF-8
done 84
85 Visiting: http://cme.uic.edu
text/html; charset=UTF-8
done 85
86 Visiting: http://cs.uic.edu
text/html; charset=utf-8
done 86
87 Visiting: http://ece.uic.edu
text/html; charset=utf-8
done 87
88 Visiting: http://mie.uic.edu
text/html; charset=UTF-8
done 88
89 Visiting: http://bioe.uic.edu
text/html; charset=UTF-8
done 89
90 Visiting: http://engineering.uic.edu/home
text/html; charset=UTF-8
done 90
91 Visiting: http://makerspace.uic.edu
text/html; charset=UTF-8
done 91
92 Visiting: http://www.engineering.uic.edu/k-teacher/janet-m-kashuba
text/html; charset=UTF-8
done 92
93 Visiting: http://www.engineering.uic.edu/home
text/html; charset=UTF-8
done 93
94 Visiting: https://www.cs.uic.edu/admitted-undergraduate-students
text/html; cha

done 136
137 Visiting: http://engineering.uic.edu/wp-content/uploads/2016/08/Form_UG_Research.pdf
application/pdf
PDF FOUND!!!
done 137
138 Visiting: http://engineering.uic.edu/wp-content/uploads/2016/08/COEpet.pdf
HTTP Error 404: Not Found
 **Failed!**
138 Visiting: http://engineering.uic.edu/wp-content/uploads/2016/08/GradCheck.pdf
application/pdf
PDF FOUND!!!
done 138
139 Visiting: http://catalog.uic.edu/ucat/academic-calendar
text/html; charset=UTF-8
done 139
140 Visiting: https://www.cs.uic.edu/registration-restrictions-for-current-cs-graduate-students
text/html; charset=utf-8
done 140
141 Visiting: https://www.cs.uic.edu/graduate-programs/cs-graduate-program-degree-requirements
text/html; charset=utf-8
done 141
142 Visiting: https://www.cs.uic.edu/~grad/2017_Graduate_Program_Requirements.pdf
application/pdf
PDF FOUND!!!
done 142
143 Visiting: https://www.cs.uic.edu/~grad/2015_Graduate_Program_Requirements.pdf
application/pdf
PDF FOUND!!!
only algorithm code 1 and 2 are supported


...

text/html; charset=UTF-8
done 2850
2851 Visiting: https://catalog.uic.edu/all-course-descriptions/socw
text/html; charset=UTF-8
done 2851
2852 Visiting: https://catalog.uic.edu/all-course-descriptions/soc
text/html; charset=UTF-8
done 2852
2853 Visiting: https://catalog.uic.edu/all-course-descriptions/span
text/html; charset=UTF-8
done 2853
2854 Visiting: https://catalog.uic.edu/all-course-descriptions/sped
text/html; charset=UTF-8
done 2854
2855 Visiting: https://catalog.uic.edu/all-course-descriptions/spec
text/html; charset=UTF-8
done 2855
2856 Visiting: https://catalog.uic.edu/all-course-descriptions/stat
text/html; charset=UTF-8
done 2856
2857 Visiting: https://catalog.uic.edu/ucat/admissions-registration-finances/student-tuition-fees
text/html; charset=UTF-8
done 2857
2858 Visiting: https://catalog.uic.edu/all-course-descriptions/sabr
text/html; charset=UTF-8
done 2858
2859 Visiting: https://catalog.uic.edu/all-course-descriptions/surg
text/html; charset=UTF-8
done 2859
2860 Visi

text/html; charset=UTF-8
done 2920
2921 Visiting: https://testing.uic.edu/clep-exams-offered-at-uic
HTTP Error 404: Not Found
 **Failed!**
2921 Visiting: http://businessconnect.uic.edu
text/html; charset=utf-8
done 2921
2922 Visiting: http://business.uic.edu/liautaud-graduate-school-business-uic
text/html; charset=utf-8
done 2922
2923 Visiting: https://businessconnect.uic.edu/student-services/academic-resources
text/html; charset=utf-8
done 2923
2924 Visiting: https://catalog.uic.edu/search/?P=MATH%20160
text/html; charset=UTF-8
done 2924
2925 Visiting: https://businessconnect.uic.edu/student-services
text/html; charset=utf-8
done 2925
2926 Visiting: https://businessconnect.uic.edu/undergraduate
text/html; charset=utf-8
done 2926
2927 Visiting: https://businessconnect.uic.edu/student-services/scholarships
text/html; charset=utf-8
done 2927
2928 Visiting: https://businessconnect.uic.edu/business-career-center
text/html; charset=utf-8
done 2928
2929 Visiting: https://businessconnect.uic.

text/html; charset=UTF-8
done 3000
3001 Visiting: https://catalog.uic.edu/search/?P=CME%20302
text/html; charset=UTF-8
done 3001
3002 Visiting: https://catalog.uic.edu/search/?P=CME%20311
text/html; charset=UTF-8
done 3002
3003 Visiting: https://catalog.uic.edu/search/?P=CME%20315
text/html; charset=UTF-8
done 3003
3004 Visiting: https://catalog.uic.edu/search/?P=EOHS%20440
text/html; charset=UTF-8
done 3004
3005 Visiting: https://catalog.uic.edu/search/?P=CME%20322
text/html; charset=UTF-8
done 3005
3006 Visiting: https://catalog.uic.edu/search/?P=EOHS%20472
text/html; charset=UTF-8
done 3006
3007 Visiting: https://catalog.uic.edu/search/?P=GEOG%20444
text/html; charset=UTF-8
done 3007
3008 Visiting: https://catalog.uic.edu/search/?P=CME%20430
text/html; charset=UTF-8
done 3008
3009 Visiting: https://catalog.uic.edu/search/?P=ME%20347
text/html; charset=UTF-8
done 3009
3010 Visiting: https://catalog.uic.edu/search/?P=CME%20401
text/html; charset=UTF-8
done 3010
3011 Visiting: https://

text/html; charset=UTF-8
done 3076
3077 Visiting: https://catalog.uic.edu/search/?P=IE%20471
text/html; charset=UTF-8
done 3077
3078 Visiting: https://catalog.uic.edu/search/?P=ME%20496
text/html; charset=UTF-8
done 3078
3079 Visiting: https://catalog.uic.edu/search/?P=ME%20497
text/html; charset=UTF-8
done 3079
3080 Visiting: https://catalog.uic.edu/search/?P=ME%20212
text/html; charset=UTF-8
done 3080
3081 Visiting: https://catalog.uic.edu/search/?P=CME%20359
text/html; charset=UTF-8
done 3081
3082 Visiting: https://catalog.uic.edu/search/?P=IE%20312
text/html; charset=UTF-8
done 3082
3083 Visiting: https://catalog.uic.edu/search/?P=ME%20205
text/html; charset=UTF-8
done 3083
3084 Visiting: https://catalog.uic.edu/search/?P=ME%20447
text/html; charset=UTF-8
done 3084
3085 Visiting: https://catalog.uic.edu/search/?P=ME%20250
text/html; charset=UTF-8
done 3085
3086 Visiting: https://catalog.uic.edu/search/?P=IE%20380
text/html; charset=UTF-8
done 3086
3087 Visiting: https://catalog.uic

text/html; charset=UTF-8
done 3158
3159 Visiting: https://catalog.uic.edu/search/?P=NURS%20414
text/html; charset=UTF-8
done 3159
3160 Visiting: https://catalog.uic.edu/search/?P=NURS%20406
text/html; charset=UTF-8
done 3160
3161 Visiting: https://catalog.uic.edu/search/?P=NUPR%20415
text/html; charset=UTF-8
done 3161
3162 Visiting: https://catalog.uic.edu/ucat/colleges-depts/nursing.pdf
HTTP Error 404: Not Found
 **Failed!**
3162 Visiting: https://pharmacy.uic.edu/programs/pharmd-professional-program
text/html; charset=UTF-8
done 3162
3163 Visiting: https://gppa.uic.edu/current-students/conditions-of-acceptance
text/html; charset=UTF-8
done 3163
3164 Visiting: https://pharmacy.uic.edu/programs/pharmd-professional-program/admission-requirements-new/prepharmacy-prerequisite-coursework-guides
text/html; charset=UTF-8
done 3164
3165 Visiting: https://pharmacy.uic.edu/programs/pharmd-professional-program/pathways-to-admissions-pharmd/transfer-students
text/html; charset=UTF-8
done 3165
316

In [5]:
dictionary.keys()

dict_keys([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219,

#### To run this cell, convert it in code cell
##write the url_dictionary into file
with io.open("url_dictionary2.txt", 'w', encoding="utf-8") as doc:
    i=1
    for document in dictionary.keys():
        doc.write(str(document) + "<id|url>" + dictionary[document] + "<element>")
        #doc.write("<value>"+str(documents_weighted_norm[document])+"</value> <element> ")
        #doc.write("\n")
        i+=1
        #if (i==10):
        #    break